In [2]:
import cv2
import numpy as np
from chainer.dataset import DatasetMixin
from PIL import Image
import chainer
import chainer.links as L
import chainer.functions as F
from chainer.optimizers import Adam
from chainer.iterators import SerialIterator

In [3]:
batch_size = 4
data_directory = 'C:\\Users\\Casper\\Documents\\Data Science 2019-2020\\NeuripsGit\\NeurIPS\\Lab7\\data' # Make a directory to store the data and enter it here.
                    # We will be using a smaller dataset (LFW) than the one used in the paper (CelebA) for computational resource considerations.
                    # Download it from http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz.
device = -1
epochs = 100
lambda_ = {'feature': 1., 'pixel': 1., 'total_variation': 1e-5}
model_directory = 'C:\\Users\\Casper\\Documents\\Data Science 2019-2020\\NeuripsGit\\NeurIPS\\Lab7\\VGG' # Make a directory to store the models and enter it here. Move Vgg4Layers.npz to the model directory.
outsize = (96, 96)

**Preprocessing functions** (taken from https://github.com/mbeyeler/opencv-python-blueprints)  
In the following cell, you will implement some of the preprocessing functions. The rest of the preprocessing steps have already been applied to the data.  
*Tasks*
- (1) Implement the resizing operation. That is, you should extract the data, resize each portrait to 96 pixels x 96 pixels and save them to the data directory as JPG.
- (2) The pencil sketch class implements the sketch effect in a simpler way than the one mentioned in the lecture. Explain how/why the used operations (blur and divide) convert portraits to sketches, and how it differs from that which was mentioned in the lecture?

In [5]:
# (1) start
for dirname, dirnames, filenames in os.walk('.'):
    

filenames = [img for img in glob.glob("images/*.jpg")]


def resize(img_rgb):
    dim = (96, 96)
    img_res = cv2.resize(img_rgb, dim, interpolation = cv2.INTER_AREA)
    return img_res


# (1) end

class PencilSketch:
    """Pencil sketch effect
        A class that applies a pencil sketch effect to an image.
    """

    def __init__(self, (width, height)):
        """Initialize parameters
            :param (width, height): Image size.
        """
        self.width = width
        self.height = height

        # try to open background canvas (if it exists)
        self.canvas = cv2.imread(bg_gray, cv2.CV_8UC1)
        if self.canvas is not None:
            self.canvas = cv2.resize(self.canvas, (self.width, self.height))

    def render(self, img_rgb):
        """Applies pencil sketch effect to an RGB image
            :param img_rgb: RGB image to be processed
            :returns: Processed RGB image
        """
        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        img_blur = cv2.GaussianBlur(img_gray, (21, 21), 0, 0)
        img_blend = cv2.divide(img_gray, img_blur, scale=256)

        # return cv2.cvtColor(img_blend, cv2.COLOR_GRAY2RGB)
        return img_blend

def pencil_sketch(img_rgb):
    pencilSketch = PencilSketch((img_rgb.shape[1], img_rgb.shape[0]))

    return pencilSketch.render(img_rgb)

# (2) Write your answer here.

def preprocess(img):
    if img.mode == 'L':
        return np.rollaxis(np.asarray(img, 'float32')[..., None], 2)
    else:
        return np.rollaxis(np.asarray(img, 'float32')[..., ::-1], 2)

IndentationError: expected an indented block (<ipython-input-5-d5f02840e5e2>, line 4)

**Data class**  
The following cell defines the data class. It is used to manage the data (loading, etc.). *You do not have to make any changes to the code.*  
*Task*
- (1) Study the code and refer to the chainer docuimentation if anything is unclear. You will be expected to write similar code by yourself in the coming practicals.

In [0]:
class Dataset(DatasetMixin):
    def __init__(self, data_files):
        self.data_files = data_files

        def __len__(self):
            return len(self.data_files)

    def get_example(self, i):
        t = np.asarray(Image.open(self.data_files[i]).convert('RGB').resize((96, 96), Image.LANCZOS), 'f').transpose(2, 0, 1)
        x = pencil_sketch(np.asarray(Image.open(self.data_files[i]).convert('RGB').resize((96, 96), Image.LANCZOS)), 'f')[None]

        return t, x

**Model classes**   
In the following cellyou will implement the model classes.  
*Tasks*   
- (1) Implement the layers of the model by filling in the missing code.
- (2) Implement the forward pass of the residual block by filling in the missing code.

In [0]:
class Model(Chain):
    def __init__(self, in_channels, outsize):
        super(Model, self).__init__()

        with self.init_scope():
            # (1) start
            # convolution2D_0 = ...
            # batchNormalization_0 = ...
            # convolution2D_1 = ...
            # batchNormalization_1 = ...
            # convolution2D_2 = ...
            # batchNormalization_2 = ...
            # residualBlock_3 = ...
            # residualBlock_4 = ...
            # residualBlock_5 = ...
            # residualBlock_6 = ...
            # residualBlock_7 = ...
            # deconvolution2D_8 = ...
            # batchNormalization_8 = ...
            # (1) end
            deconvolution2D_9 = L.Deconvolution2D(64, 32, 3, 2, 1, True, outsize)
            batchNormalization_9 = L.BatchNormalization(32)
            convolution2D_10 = L.Convolution2D(32, 3, 9, pad = 4, nobias = True)
            batchNormalization_10 = L.BatchNormalization(3)

        self.in_channels = in_channels
        self.outsize = outsize

    def __call__(self, x, finetune = False):
        h = self.convolution2D_0(x)
        h = self.batchNormalization_0(h, finetune)
        h = F.relu(h)
        h = self.convolution2D_1(h)
        h = self.batchNormalization_1(h, finetune)
        h = F.relu(h)
        h = self.convolution2D_2(h)
        h = self.batchNormalization_2(h, finetune)
        h = F.relu(h)
        h = self.residualBlock_3(h, finetune)
        h = self.residualBlock_4(h, finetune)
        h = self.residualBlock_5(h, finetune)
        h = self.residualBlock_6(h, finetune)
        h = self.residualBlock_7(h, finetune)
        h = self.deconvolution2D_8(h)
        h = self.batchNormalization_8(h, finetune)
        h = F.relu(h)
        h = self.deconvolution2D_9(h)
        h = self.batchNormalization_9(h, finetune)
        h = F.relu(h)
        h = self.convolution2D_10(h)
        h = self.batchNormalization_10(h, finetune)
        y = 127.5 * F.tanh(h) + 127.5

        return y

class ResidualBlock(Chain):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()

        with self.init_scope():
            batchNormalization_0 = L.BatchNormalization(out_channels)
            batchNormalization_1 = L.BatchNormalization(out_channels)
            convolution2D_0 = L.Convolution2D( in_channels, out_channels, 3, pad = 1, nobias = True)
            convolution2D_1 = L.Convolution2D(out_channels, out_channels, 3, pad = 1, nobias = True)

        self.in_channels = in_channels
        self.out_channels = out_channels

    def __call__(self, x, finetune = False):
        # (2) start
        # h = ...
        # h = ...
        # h = ...
        # h = ...
        # h = ...
        # y = ...
        # (2) start

        return y

**Loss classes**  
In the following cell, you will implement the loss classes.  
*Tasks*  
- (1) You are provided with a custom VGG-16 implementation. How does it differ than the original implementation? Why can we get away with using the simpler implementation?
- (2) (Answer is given) Implement the missing convolution layer of the total variation loss by filling in the missing code.
- (3) Implement the forward pass of the total variation loss by filling in the missing code.
- (4) Implement the feature loss component in the forward pass of the loss function by filling in the missing code.
- (5) Explain why the loss components are scaled.
- (6) Explain why the target features are extracted in test mode.

In [0]:
class Vgg4Layers(Chain):
    def __init__(self):
        super(Vgg4Layers, self).__init__()

        with self.init_scope():
            convolution2D_0 = L.Convolution2D(  3,  64, 3, pad = 1)
            convolution2D_1 = L.Convolution2D( 64,  64, 3, pad = 1)
            convolution2D_2 = L.Convolution2D( 64, 128, 3, pad = 1)
            convolution2D_3 = L.Convolution2D(128, 128, 3, pad = 1)

        add_persistent('mean', np.array([[[[103.939]], [[116.779]], [[123.68]]]]), 'float32')

    def __call__(self, x):
        h = x - F.broadcast_to(self.mean, x.shape)
        h = self.convolution2D_0(h)
        h = F.relu(h)
        h = self.convolution2D_1(h)
        h = F.relu(h)
        h = F.max_pooling_2d(h, 2, 2)
        h = self.convolution2D_2(h)
        h = F.relu(h)
        h = self.convolution2D_3(h)
        y = F.relu(h)

        return y

class TotalVariationLoss(Chain):
    def __init__(self):
        super(TotalVariationLoss, self).__init__()

        with self.init_scope():
            convolution2D_0 = L.Convolution2D(3, 1, 2, nobias = True, initialW = np.array([3 * [[[-1], [1]]]], 'float32'))
            # (2) start
            convolution2D_1 = L.Convolution2D(3, 1, 2, nobias = True, initialW = np.array([3 * [[[-1,   1]]]], 'float32'))
            # (2) end

    def __call__(self, x):
        # (3) start
        # (3) end

        return y
    
    class LossFunction(object):
    def __init__(self, lambda_):
        self.totalVariationLoss = TotalVariationLoss()
        self.vgg4Layers         = Vgg4Layers()

        def __call__(t, y):            
            with chainer.using_config('train', False):
                t_ = self.vgg4Layers(t)

            # (4) start
            # (4) end
            pixel_loss = lambda_['pixel'] * F.mean_squared_error(t , y) + 
            total_variation_loss = lambda_['total_variation'] * self.totalVariationLoss(t, y)
            loss = feature + pixel_loss + total_variation_loss

            return loss

**Initialization**  
The following cell initializes the loss function, the loss history, the model, the optimizer, the datasets and the iterators. *You do not have to make any changes to the code.*  
*Tasks*
- (1) Study the code and refer to the chainer docuimentation if anything is unclear. You will be expected to write similar code by yourself in the coming practicals. 
- (2) What are the boolean arguments that are passed to the SerialIterator class?  
- (3) Why is it false for the training iterator but not for other iterators? In other words, what would happen if we were to set it to false for the training iterator and true for the other iterators?

In [0]:
lossFunction = LossFunction(lambda_)
load_npz('{:s}/Vgg4Layers.npz'.format(model_directory, epoch), lossFunction.vgg4Layers)
loss_history = {'training': [], 'validation': []}
model = Model() if device < 0 else Model().to_gpu(device)
optimizer = Adam()

optimizer.setup(model)

data_file = sorted(glob('{}/*.jpg'.format(data_directory)))
training_set = Dataset(data_file[int(:.64 * len(data_file))])
validation_set = Dataset(data_file[int(.64 * len(data_file) : .8 * len(data_file))])
test_set = Dataset(data_file[int(.8 * len(data_file) :)])
training_iterator = SerialIterator(training_set, batch_size, False, True)
test_iterator = SerialIterator(validation_set, batch_size, False, False)
validation_iterator = SerialIterator(test_set , batch_size, False, False)

**Training and validation**  
In the following cell, you will train and validate your model.
*Tasks*   
- (1) Implement training loss estimation, backprop and parameter update.
- (2) (Answer is given) Implement validation loss history
- (3) (Answer is given) Implement model serialization

In [0]:
for i in range(epochs):
    loss_history['training'].append(0)

    for j, batch in enumerate(training_iterator):
        with chaine.using_config('train', True):
            t, x = concat_examples(batch, device)
            y = model(x)
            # (1) start
            # (1) end

        loss_history['training'][-1] += float(loss.data)

    loss_history['training'][-1] /= j + 1
    # (2) start
    loss_history['validation'].append(0)

    for j, batch in enumerate(validation_iterator):
        with chaine.using_config('train', False):
            t, x = concat_examples(batch, device)
            y = model(x)
            loss = lossFunction(t, y)

        loss_history['validation'][-1] += float(loss.data)

    loss_history['validation'][-1] /= j + 1
    # (2) end
    print('epoch: {:3d} / {:03d}, training loss: {:.4f}, validation loss: {:.4f}.'.format(i + 1, epochs, log['training'], log['validation']))
    np.savez('{:s}/loss_history_{:03d}.npz'.format(model_directory, epoch), loss_history)
    # (3) start
    save_npz('{:s}/model_{:03d}.npz'.format(model_directory, epoch), model)
    # (3) end
    save_npz('{:s}/optimizer_{:03d}.npz'.format(model_directory, epoch), optimizer)

**Test**  
In the following cell, you will test your model.  
*Tasks*
- (1) Estimate the test loss, print it and save it.
- (2) Estimate the validation metrics, print them and save them (tip: scikit-image)
- (3) Plot example results (i.e., plot a few t, x and y)
- (4) Dicuss your implementation in 300 - 350 words (e.g., how good your results are, how you can improve your model, etc.)

In [0]:
# (1) start
# (1) end

# (2) and (3) start
# (2) can be implementted as above using scikitt-image functions (i.e., replace lossFunction with them)
# (3) can be implementted with PIL, matplotlib or any other imaging library (just plot t, x, y after transposing them and casting them to uint8)
# (2) and (3) end